In [33]:
# To do analysis on multiple stocks

In [34]:
# libraries 

from getData import get_data, get_ohlcv
from technical_indicator import LR, MACD, RSI, BB, ATR, ADX, RENKO, anomaly, historical_volatility, get_support_resistance_levels, VWAP
from Returns import cummulative_returns, CAGR, volatility, SHARPE, SORTINO, MAXDROWDOWN, CALMAR
from signals import stratgey1_mean_reversion, label_sd_with_signals,label_exit_signals


from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import copy
import os

%load_ext autoreload
%autoreload 2
pd.set_option('mode.chained_assignment', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Getting Data

In [35]:
tickers = [
    "NVDA", "MSFT", "AAPL", "AMZN", "META", "AVGO", "GOOGL", "TSLA", "BRK-B", "GOOG",
    "JPM", "LLY", "V", "XOM", "COST", "MA", "UNH", "NFLX", "WMT", "PG",
    "JNJ", "HD", "ABBV", "BAC", "CRM", "ORCL", "CSCO", "GE", "PLTR", "IBM",
    "T", "PFE", "KO", "PM", "QCOM", "ACN", "TMO", "MCD", "VZ", "NKE",
    "LIN", "MRK", "ABT", "AMD", "ADBE", "AMAT", "LOW", "DHR", "TXN", "UPS",
    "NEE", "INTU", "SPGI", "MS", "CAT", "UNP", "CVX", "GS", "RTX", "ISRG",
    "AMGN", "INTC", "BLK", "MDT", "DE", "ADI", "NOW", "SCHW", "LRCX", "C",
    "SYK", "MO", "TGT", "MMC", "CI", "ZTS", "ADP", "ELV", "ETN", "PANW",
    "PYPL", "REGN", "BDX", "WM", "SLB", "APD", "FISV", "VRTX", "CB", "SO",
    "CL", "EOG", "DUK", "PGR", "MNST", "HUM", "FCX", "AON", "PSX", "TRV"
]

In [36]:
# Store data in dictionary
ohlcv_dict = {}
for stock in tickers:
    ohlcv_dict[stock] = get_ohlcv(ticker=stock, period = '1y', interval='1d', multi_level_index=False)
    # ohlcv_dict[stock] = ohlcv_dict[stock].between_time('09:35', '16:00')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [37]:
data = copy.deepcopy(ohlcv_dict)

In [38]:
for stock_name in tickers:
    try:
        cummulative_returns(data[stock_name])
        ATR(data[stock_name])
        anomaly(data[stock_name])
        support, resistance = get_support_resistance_levels(data[stock_name])
        data[stock_name]['support'] = support
        data[stock_name]['resistance'] = resistance
        historical_volatility(data[stock_name])
        # historical_volatility called twice? Removed duplicate call
        RSI(data[stock_name])
        VWAP(data[stock_name])
        BB(data[stock_name])
        data[stock_name]['Close'] = round(data[stock_name]['Close'], 2)
        data[stock_name]['Signal'] = data[stock_name].apply(stratgey1_mean_reversion, axis=1)
        data[stock_name]['m_half_50'] = data[stock_name]['Close'].ewm(halflife=50).mean()
        data[stock_name]['Distance_From_Mean'] = data[stock_name]['Close'] - data[stock_name]['m_half_50']
        data[stock_name]=label_sd_with_signals(data[stock_name])
        data[stock_name]=label_exit_signals(data[stock_name])
        print(f'Now processing {stock_name}')
        
    except KeyError:
        print(f"Data for {stock_name} not found, skipping.")
        continue
    except Exception as e:
        print(f"Error processing {stock_name}: {e}, skipping.")
        continue


Now processing NVDA
Now processing MSFT
Now processing AAPL
Now processing AMZN
Now processing META
Now processing AVGO
Now processing GOOGL
Now processing TSLA
Now processing BRK-B
Now processing GOOG
Now processing JPM
Now processing LLY
Now processing V
Now processing XOM
Now processing COST
Now processing MA
Now processing UNH
Now processing NFLX
Now processing WMT
Now processing PG
Now processing JNJ
Now processing HD
Now processing ABBV
Now processing BAC
Now processing CRM
Now processing ORCL
Now processing CSCO
Now processing GE
Now processing PLTR
Now processing IBM
Now processing T
Now processing PFE
Now processing KO
Now processing PM
Now processing QCOM
Now processing ACN
Now processing TMO
Now processing MCD
Now processing VZ
Now processing NKE
Now processing LIN
Now processing MRK
Now processing ABT
Now processing AMD
Now processing ADBE
Now processing AMAT
Now processing LOW
Now processing DHR
Now processing TXN
Now processing UPS
Now processing NEE
Now processing INTU
N

In [39]:
# data['AAPL']

In [40]:
buy_opportunities = []


for stock_name in tickers:
    try:
        df = data[stock_name]
        df['Date'] = df.index
        # filtered = df[df['Label'].isin(['BELOW 1 SD'])].copy()
        filtered = df[df['Label'].isin(['BELOW 2 SD', 'BELOW 3 SD'])].copy()
        # filtered = df[df['Label'].isin(['BELOW 3 SD'])].copy()
        filtered['Stock'] = stock_name  # Optional: add stock name column
        buy_opportunities.append(filtered)
    except KeyError:
        print(f"Data for {stock_name} not found, skipping.")
        continue
    except Exception as e:
        print(f"Error processing {stock_name}: {e}, skipping.")

buy_opportunity = pd.concat(buy_opportunities, ignore_index=True)


Data for FISV not found, skipping.


In [44]:
track = buy_opportunity.sort_values(by='Date', ascending=False)
track[track['Date']>'2025-07-01']['Stock'].unique()

array(['PGR', 'ABT', 'ELV', 'PG'], dtype=object)

In [42]:
# buy_opportunity['Stock'].unique()

In [43]:
track[track['Date']>'2025-05-01']['Stock'].unique()


array(['PGR', 'ABT', 'ELV', 'PG', 'MCD', 'UNH', 'BDX', 'MRK'],
      dtype=object)